In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import cv2 as cv
import pandas as pd
import os
import pathlib
import sys
import numpy as np
import random
import shutil
import cv2 as cv
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
# from tensorflow.keras.utils import np_utils
from sklearn import metrics
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
IMAGE_SIZE = 256
pathtxt = pathlib.Path('pathtxt/')
data = pathlib.Path('datasetproject/')
train_data = pathlib.Path('datasetproject/train')
test_data = pathlib.Path('datasetproject/test')

In [ ]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory)
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f)
                      for f in os.listdir(label_directory)]
        for f in file_names:
            img = cv.imread(f)
            img = cv.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
            images.append(img)
            labels.append(int(d))

    images, labels = np.asarray(images), np.asarray(labels)

    return images, labels


In [ ]:
num_folds = 10

num_class = 6
IMAGE_SIZE = 256
epoch = 70
loss_function = sparse_categorical_crossentropy
optmizer = SGD(learning_rate=0.1, momentum=0.9)
batchsize = 32

# container for metrics
acc_folds = []
f1_folds = []
prec_folds = []
recall_folds = []

In [ ]:
def xception(num_class):
    stride = (2, 2)
    kernel_size = (3, 3)
    pool_size = (2, 2)
    Channel_axis = 3

    def entry_flow(img_input):

        x = layers.Conv2D(32, kernel_size=kernel_size,
                          use_bias=False, kernel_initializer='glorot_uniform',
                          strides=stride, padding='SAME')(img_input)
        x = layers.BatchNormalization(axis=Channel_axis)(x)
        x = LeakyReLU(alpha=0.1)(x)

        x = layers.Conv2D(64, kernel_size=kernel_size,
                          use_bias=False, kernel_initializer='glorot_uniform',
                          padding='SAME')(x)
        x = layers.BatchNormalization(axis=Channel_axis)(x)
        x = LeakyReLU(alpha=0.1)(x)

        x_temp = x

        for filter in [128, 256, 728]:
            if filter != 128:
                x = LeakyReLU(alpha=0.1)(x)
            x = layers.SeparableConv2D(filters=filter, kernel_size=kernel_size,
                                       use_bias=False, kernel_initializer='glorot_uniform',
                                       padding='SAME')(x)
            x = layers.BatchNormalization(axis=Channel_axis)(x)

            x = LeakyReLU(alpha=0.1)(x)
            x = layers.SeparableConv2D(filters=filter, kernel_size=kernel_size,
                                       use_bias=False, kernel_initializer='glorot_uniform',
                                       padding='SAME')(x)
            x = layers.BatchNormalization(axis=Channel_axis)(x)

            x = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

            x_shortcut = layers.Conv2D(filters=filter, kernel_size=(1, 1),
                                       use_bias=False, kernel_initializer='glorot_uniform',
                                       strides=stride, padding='SAME')(x_temp)
            x_shortcut = layers.BatchNormalization(axis=Channel_axis)(x_shortcut)

            x = layers.add([x, x_shortcut])
            x_temp = x

        return x

    def middle_flow(x):

        x_temp = x

        for i in range(8):
            x = LeakyReLU(alpha=0.1)(x)
            x = layers.SeparableConv2D(filters=728, kernel_size=kernel_size,
                                       use_bias=False, kernel_initializer='glorot_uniform',
                                       padding='SAME')(x)
            x = layers.BatchNormalization(axis=Channel_axis)(x)

            x = LeakyReLU(alpha=0.1)(x)
            x = layers.SeparableConv2D(filters=728, kernel_size=kernel_size,
                                       use_bias=False, kernel_initializer='glorot_uniform',
                                       padding='SAME')(x)
            x = layers.BatchNormalization(axis=Channel_axis)(x)

            x = LeakyReLU(alpha=0.1)(x)
            x = layers.SeparableConv2D(filters=728, kernel_size=kernel_size,
                                       use_bias=False, kernel_initializer='glorot_uniform',
                                       padding='SAME')(x)
            x = layers.BatchNormalization(axis=Channel_axis)(x)

            x = layers.add([x, x_temp])
            x_temp = x

        return x

    def exit_flow(x):

        x_temp = x

        x = LeakyReLU(alpha=0.1)(x)
        x = layers.SeparableConv2D(filters=728, kernel_size=kernel_size,
                                   use_bias=False, kernel_initializer='glorot_uniform',
                                   padding='SAME')(x)
        x = layers.BatchNormalization(axis=Channel_axis)(x)

        x = LeakyReLU(alpha=0.1)(x)
        x = layers.SeparableConv2D(filters=1024, kernel_size=kernel_size,
                                   use_bias=False, kernel_initializer='glorot_uniform',
                                   padding='SAME')(x)
        x = layers.BatchNormalization(axis=Channel_axis)(x)

        x = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

        x_shortcut = layers.Conv2D(filters=1024, kernel_size=kernel_size,
                                   use_bias=False, kernel_initializer='glorot_uniform',
                                   strides=stride, padding='SAME')(x_temp)
        x_shortcut = layers.BatchNormalization(axis=Channel_axis)(x_shortcut)

        x = layers.add([x, x_shortcut])

        x = layers.SeparableConv2D(filters=1536, kernel_size=kernel_size,
                                   use_bias=False, kernel_initializer='glorot_uniform',
                                   padding='SAME')(x)
        x = layers.BatchNormalization(axis=Channel_axis)(x)
        x = LeakyReLU(alpha=0.1)(x)

        x = layers.SeparableConv2D(filters=2048, kernel_size=kernel_size,
                                   use_bias=False, kernel_initializer='glorot_uniform',
                                   padding='SAME')(x)
        x = layers.BatchNormalization(axis=Channel_axis)(x)
        x = LeakyReLU(alpha=0.1)(x)

        x = layers.GlobalAveragePooling2D()(x)

        #         x = layers.Dropout(0.8)(x)

        output = layers.Dense(num_class, activation=tf.nn.softmax, dtype=tf.float32)(x)

        return output

    img_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    x = entry_flow(img_input)
    x = middle_flow(x)
    output = exit_flow(x)

    model = models.Model(inputs=img_input, outputs=output, name='Xception')

    return model

In [ ]:
imagens, labels = load_data(str(train_data))

# Cross fold validation

kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=29)

for train, test in kf.split(imagens, labels):
    imgs_train, imgs_val = imagens[train], imagens[test]
    labs_train, labs_val = labels[train], labels[test]

    model = None
    model = xception(num_class)
    model.compile(loss=loss_function,
                      optimizer=optmizer,
                      metrics=['accuracy'])

    xcept = model.fit(imgs_train, labs_train,
                      batch_size=batchsize,
                      validation_data=(imgs_val, labs_val), verbose=1,
                      epochs=epoch)
    
    imgs_test, labs_test = load_data(test_data)

    labs_predict = model.predict(imgs_test)
    labs_predict = np.argmax(labs_predict, axis=1)

    accuracy = metrics.accuracy_score(labs_test, labs_predict)
    acc_folds.append(accuracy)

    f1 = metrics.f1_score(labs_test, labs_predict, average="macro")
    f1_folds.append(f1)

    precision = metrics.precision_score(labs_test, labs_predict, average="macro")
    prec_folds.append(precision)

    recall = metrics.recall_score(labs_test, labs_predict, average="macro")
    recall_folds.append(recall)

    print('Acurracy: %f' % accuracy)
    print('F1: %f' % f1)
    print('Precision: %f' % precision)
    print('Recall: %f' % recall)

Epoch 1/70
 4/18 [=====>........................] - ETA: 8:07 - loss: 2.9220 - accuracy: 0.2188